#### Settings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
%cd /content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Raw

/content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Raw


In [ ]:
nta_2020_df = pd.read_csv('2020nta.csv')

#### Tree Canopy Change


In [ ]:
%cd /content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified

/content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified


In [ ]:
gain_df = pd.read_excel("NYC_TreeCanopyCha_Gain_TableToExcel.xlsx")
nochange_df = pd.read_excel("NYC_TreeCanopyCha_Nochange_TableToExcel.xlsx")
loss_df = pd.read_excel("NYC_TreeCanopyCha_Loss_TableToExcel.xlsx")

# Convert 'Shape_Area' in nta_2020_df to numeric after removing commas
nta_2020_df['Shape_Area'] = nta_2020_df['Shape_Area'].replace(',', '', regex=True).astype(float)

# Normalize tree canopy areas by NTA area
# Merge each tree canopy dataframe with nta_2020_df to perform the division
gain_df = gain_df.merge(nta_2020_df[['NTAName', 'Shape_Area']], left_on='ntaname', right_on='NTAName', how='left')
gain_df['calculate_gain'] = gain_df['SUM_Shape_Area'] / gain_df['Shape_Area']

nochange_df = nochange_df.merge(nta_2020_df[['NTAName', 'Shape_Area']], left_on='ntaname', right_on='NTAName', how='left')
nochange_df['calculate_nochange'] = nochange_df['SUM_Shape_Area'] / nochange_df['Shape_Area']

loss_df = loss_df.merge(nta_2020_df[['NTAName', 'Shape_Area']], left_on='ntaname', right_on='NTAName', how='left')
loss_df['calculate_loss'] = loss_df['SUM_Shape_Area'] / loss_df['Shape_Area']

#Issue due to Shape_Area values being formatted as strings with commas, which prevents the division operation.
#remove commas and convert the Shape_Area column to a numeric data type.

In [ ]:
gain_df.head()

,OBJECTID,ntaname,nta2020,FREQUENCY,SUM_Shape_Area,NTAName,Shape_Area,calculate_gain
0,1,NaN,NaN,2387,6.308119e+03,NaN,NaN,NaN
1,2,Allerton,BX1104,9888,7.373891e+04,Allerton,1.575392e+07,0.004681
2,3,Alley Pond Park,QN1191,10178,1.526862e+05,Alley Pond Park,2.613240e+07,0.005843
3,4,Annadale-Huguenot-Prince's Bay-Woodrow,SI0304,108697,1.106465e+06,Annadale-Huguenot-Prince's Bay-Woodrow,1.804263e+08,0.006133
4,5,Arden Heights-Rossville,SI0303,42625,3.916532e+05,Arden Heights-Rossville,6.777396e+07,0.005779


In [ ]:
# Apply weights to calculate the weighted average
# Add the weighted values to each respective DataFrame
gain_df['weighted_gain'] = gain_df['calculate_gain'] * 1  # +1 for gain
nochange_df['weighted_nochange'] = nochange_df['calculate_nochange'] * 1  # +1 for no change
loss_df['weighted_loss'] = loss_df['calculate_loss'] * -1  # -1 for loss

# Merge the weighted values into the NTA dataset
weighted_df = nta_2020_df[['NTAName']].copy()
weighted_df = weighted_df.merge(gain_df[['ntaname', 'weighted_gain']], left_on='NTAName', right_on='ntaname', how='left')
weighted_df = weighted_df.merge(nochange_df[['ntaname', 'weighted_nochange']], left_on='NTAName', right_on='ntaname', how='left')
weighted_df = weighted_df.merge(loss_df[['ntaname', 'weighted_loss']], left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing values with 0
weighted_df[['weighted_gain', 'weighted_nochange', 'weighted_loss']] = weighted_df[['weighted_gain', 'weighted_nochange', 'weighted_loss']].fillna(0)

# Calculate the final weighted average
weighted_df['vulnerability_score'] = (weighted_df['weighted_gain'] + weighted_df['weighted_nochange'] + weighted_df['weighted_loss']) / 3

# Merge the calculated scores back into the original NTA dataset
nta_2020_df = nta_2020_df.merge(weighted_df[['NTAName', 'vulnerability_score']], on='NTAName', how='left')

# Fill any missing vulnerability scores with 0
nta_2020_df['vulnerability_score'] = nta_2020_df['vulnerability_score'].fillna(0)

# Save
nta_2020_df.to_excel("nta_with_treecanopy_vulnerability_scores.xlsx", index=False)


#### SLR 2050_500yr

In [ ]:
slr2050_500_df = pd.read_excel('c2050_500yr_Proje_Statistics_TableToExcel_up_noz.xlsx')

# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, slr2050_500_df, left_on='NTAName', right_on='ntaname', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('slr2050_500yr_nta2020.xlsx', index=False)


#### SLR 2050_100yr

In [ ]:
slr2050_100_df = pd.read_excel('c2050_100yr_Proje_Statistics_TableToExcel_up_noz.xlsx')

# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, slr2050_100_df, left_on='NTAName', right_on='ntaname', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('slr2050_100yr_nta2020.xlsx', index=False)

print("Export completed")

Export completed


#### SLR 2020_500yr

In [ ]:
slr2020_500_df = pd.read_excel('c2020_500yr_Proje_Statistics_TableToExcel_up_noz.xlsx')

# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, slr2020_500_df, left_on='NTAName', right_on='ntaname', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('slr2020_500yr_nta2020.xlsx', index=False)

print("Export completed")

Export completed


#### SLR 2020_100yr

In [ ]:
%cd /content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified

/content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified


In [ ]:
slr2020_100_df = pd.read_excel('c2020_100yr_Proje_Statistics_TableToExcel_up_noz.xlsx')

# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, slr2020_100_df, left_on='NTAName', right_on='ntaname', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('slr2020_100yr_nta2020.xlsx', index=False)

print("Export completed")

Export completed


#### Street Tree Sidewalk

In [ ]:
steetpav_df = pd.read_excel('streettree_Inters_Statistics_sidewalk_TableToExcel.xlsx')

# Identify missing NTAs in the building dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(steetpav_df['ntaname'])

# Create a dataframe to display missing NTAs
missing_ntas_df = pd.DataFrame(list(missing_ntas), columns=['Missing_NTAs'])
missing_ntas_df.head()

# Assign values to LSTSTATYPE
# Define the mapping: 1 for constructed/permitted/under construction, 0 for marked/blanks
value_mapping = {
    'Damage': 1,
    'NoDamage': 0,
    '': 0  # Blank rows
}

# Apply the mapping to the building dataset
steetpav_df['Value'] = steetpav_df['sidewalk'].map(value_mapping)

# calculate vulnerability value for each NTA
# Multiply 'Value' by 'FREQUENCY' and group by 'Geography' (NTA name) to sum the values
steetpav_df['Weighted_Value'] = steetpav_df['Value'] * steetpav_df['FREQUENCY']

# Group by Geography to calculate the total vulnerability value for each NTA
vulnerability_df = steetpav_df.groupby('ntaname')['Weighted_Value'].sum().reset_index()

# Merge the vulnerability values back with the NTA dataset to include all NTAs
final_df = pd.merge(nta_2020_df[['NTAName']], vulnerability_df, left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing vulnerability values with 0
final_df['Weighted_Value'].fillna(0, inplace=True)

# Export the final result to Excel
final_df.to_excel('ST_sidewalk_recalculated_values_nta2020_new.xlsx', index=False)

print("Vulnerability values calculated and exported.")

Vulnerability values calculated and exported.


<ipython-input-31-9a4e0da1debf>:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Weighted_Value'].fillna(0, inplace=True)


#### Street Tree Status

In [ ]:
steetpav_df = pd.read_excel('streettree_Inters_Statistics_status_TableToExcel.xlsx')

# Identify missing NTAs in the building dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(steetpav_df['ntaname'])

# Create a dataframe to display missing NTAs
missing_ntas_df = pd.DataFrame(list(missing_ntas), columns=['Missing_NTAs'])
missing_ntas_df.head()

# Assign values to LSTSTATYPE
# Define the mapping: 1 for constructed/permitted/under construction, 0 for marked/blanks
value_mapping = {
    'Alive': 0,
    'Dead': 1,
    'Stump': 0,
    '': 0  # Blank rows
}

# Apply the mapping to the building dataset
steetpav_df['Value'] = steetpav_df['status'].map(value_mapping)

# Calculate vulnerability value for each NTA
# Multiply 'Value' by 'FREQUENCY' and group by 'Geography' (NTA name) to sum the values
steetpav_df['Weighted_Value'] = steetpav_df['Value'] * steetpav_df['FREQUENCY']

# Group by Geography to calculate the total vulnerability value for each NTA
vulnerability_df = steetpav_df.groupby('ntaname')['Weighted_Value'].sum().reset_index()

# Merge the vulnerability values back with the NTA dataset to include all NTAs
final_df = pd.merge(nta_2020_df[['NTAName']], vulnerability_df, left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing vulnerability values with 0 (in case an NTA has no building data)
final_df['Weighted_Value'].fillna(0, inplace=True)

# Export the final result to Excel
final_df.to_excel('ST_status_recalculated_values_nta2020_new.xlsx', index=False)



print("Vulnerability values calculated and exported.")

Vulnerability values calculated and exported.


<ipython-input-29-7a818d03be33>:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Weighted_Value'].fillna(0, inplace=True)


#### Street Tree Health

In [ ]:
steetpav_df = pd.read_excel('streettree_Inters_Statistics_health_TableToExcel.xlsx')

# Identify missing NTAs in the building dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(steetpav_df['ntaname'])

# Create a dataframe to display missing NTAs
missing_ntas_df = pd.DataFrame(list(missing_ntas), columns=['Missing_NTAs'])
missing_ntas_df.head()

# Assign values to LSTSTATYPE
# Define the mapping: 1 for constructed/permitted/under construction, 0 for marked/blanks
value_mapping = {
    'Fair': 0.5,
    'Good': 0,
    'Poor': 1,
    '': 0.5  # Blank rows
}

# Apply the mapping to the building dataset
steetpav_df['Value'] = steetpav_df['health'].map(value_mapping)

# Calculate vulnerability value for each NTA
# Multiply 'Value' by 'FREQUENCY' and group by 'Geography' (NTA name) to sum the values
steetpav_df['Weighted_Value'] = steetpav_df['Value'] * steetpav_df['FREQUENCY']

# Group by Geography to calculate the total vulnerability value for each NTA
vulnerability_df = steetpav_df.groupby('ntaname')['Weighted_Value'].sum().reset_index()

# Merge the vulnerability values back with the NTA dataset to include all NTAs
final_df = pd.merge(nta_2020_df[['NTAName']], vulnerability_df, left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing vulnerability values with 0 (in case an NTA has no building data)
final_df['Weighted_Value'].fillna(0, inplace=True)

# Export the final result to Excel
final_df.to_excel('ST_health_recalculated_values_nta2020_new.xlsx', index=False)


print("Vulnerability values calculated and exported.")

Vulnerability values calculated and exported.


<ipython-input-28-489c09544bb0>:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Weighted_Value'].fillna(0, inplace=True)


In [ ]:
steetpav_df['Value']

,Value
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
911,NaN
912,NaN
913,NaN
914,NaN


#### Park Forest Restoration

In [ ]:
park_forest_df = pd.read_excel('parkforest_Projec_Statistics_TableToExcel.xlsx')

# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, park_forest_df, left_on='NTAName', right_on='ntaname', how='left')

#Export the final dataframe to an Excel file
merged_df.to_excel('park_forest_nta2020.xlsx', index=False)

print("Export completed")

Export completed


#### Street Pavement (new)

In [ ]:
steetpav_df = pd.read_excel('strt_pave_Project_Statistics_nta2020_withIntersect_TableToExcel.xlsx')

# Identify missing NTAs in the building dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(steetpav_df['ntaname'])

# Create a dataframe to display missing NTAs
missing_ntas_df = pd.DataFrame(list(missing_ntas), columns=['Missing_NTAs'])
missing_ntas_df.head()

# Assign values to LSTSTATYPE
# Define the mapping: 1 for constructed/permitted/under construction, 0 for marked/blanks
value_mapping = {
    'FAIR': 0.5,
    'GOOD': 0,
    'NR': 0.5,
    'POOR': 1,
    '': 0  # Blank rows
}

# Apply the mapping to the building dataset
steetpav_df['Value'] = steetpav_df['rating_wor'].map(value_mapping)

#  Calculate vulnerability value for each NTA
# Multiply 'Value' by 'FREQUENCY' and group by 'Geography' (NTA name) to sum the values
steetpav_df['Weighted_Value'] = steetpav_df['Value'] * steetpav_df['FREQUENCY']

# Group by Geography to calculate the total vulnerability value for each NTA
vulnerability_df = steetpav_df.groupby('ntaname')['Weighted_Value'].sum().reset_index()

# Merge the vulnerability values back with the NTA dataset to include all NTAs
final_df = pd.merge(nta_2020_df[['NTAName']], vulnerability_df, left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing vulnerability values with 0 (in case an NTA has no building data)
final_df['Weighted_Value'].fillna(0, inplace=True)

# Export the final result to Excel
final_df.to_excel('Street_pavement_recalculated_values_nta2020_new.xlsx', index=False)


print("Vulnerability values calculated and exported.")

Vulnerability values calculated and exported.


<ipython-input-19-bfae8c7c05f0>:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Weighted_Value'].fillna(0, inplace=True)


#### Wetlands

In [ ]:
wetlands_df = pd.read_excel('wetlands_Project__Statistics_TableToExcel.xlsx')

# merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, wetlands_df, left_on='NTAName', right_on='ntaname', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('wetlands_nta2020.xlsx', index=False)

print("Export completed")

Export completed


#### Parks Wild

In [ ]:
#merge with whole nta
parks_wild_df = pd.read_excel('parks_wild_Projec_Statistics_TableToExcel.xlsx')

#Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, parks_wild_df, left_on='NTAName', right_on='ntaname', how='left')

#Export the final dataframe to an Excel file
merged_df.to_excel('parks_wild_nta2020.xlsx', index=False)

print("Export completed")

Export completed


#### forest restoration

In [ ]:
%cd /content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified

/content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified


In [ ]:
import pandas as pd

forest_df = pd.read_excel('forestrestoration_Project2020_TableToExcel.xlsx')

# Identify NTAs that are in the NTA dataset but missing in the forest restoration dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(forest_df['ntaname'])


# Group by 'ntaname' and sum 'Shape_Area_park' for each NTA
park_area_sum = forest_df.groupby('ntaname').agg(
    total_park_area=('Shape_Area_park', 'sum'),
    nta_area=('Shape_Area_nta', 'first')  # Assuming 'Shape_Area_nta' is the same for each NTA
).reset_index()

# Merge with the NTA dataset to ensure all NTAs are included
merged_df = nta_2020_df[['NTAName']].merge(
    park_area_sum,
    left_on='NTAName',
    right_on='ntaname',
    how='left'
)

# Fill in missing values for NTAs without park areas
merged_df['total_park_area'].fillna(0, inplace=True)
merged_df['nta_area'].fillna(merged_df['nta_area'].mean(), inplace=True)

# Export the result to an Excel file
merged_df[['NTAName', 'total_park_area', 'nta_area']].to_excel('nta_park_area_summary_all_ntas.xlsx', index=False)

print("Exported NTA park area summary to 'nta_park_area_summary_all_ntas.xlsx'.")



Exported NTA park area summary to 'nta_park_area_summary_all_ntas.xlsx'.


<ipython-input-9-3bf7e4a44883>:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['total_park_area'].fillna(0, inplace=True)
<ipython-input-9-3bf7e4a44883>:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

#### race and ethnicity

In [ ]:
race_df = pd.read_csv('NYC EH Data Portal - Race and ethnicity (filtered).csv')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
race_df['Geography'] = race_df['Geography'].str.strip()

# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(race_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

# Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
walked_df_avg = race_df.groupby('Geography').apply(
    lambda x: x['Non-white (percent)'].mean()
).reset_index(name='Average_Non-white (percent)')

# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, walked_df_avg, left_on='NTAName', right_on='Geography', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('race_ethinicity_2020_df.xlsx', index=False)

print("Export completed: 'walked_2020_df.xlsx'")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

<ipython-input-7-4d3047495cca>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  walked_df_avg = race_df.groupby('Geography').apply(


Export completed: 'walked_2020_df.xlsx'


#### Wetlands

In [ ]:
wetlands_df = pd.read_excel('wetlands_Project_Statistics_TableToExcel.xlsx')


# Identify NTAs that are in the NTA dataset but missing in the wetlands dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(wetlands_df['ntaname'])

In [ ]:
# export the NTA names and wetlands area
# Merge the wetlands data with the NTA data
merged_df = nta_2020_df[['NTAName', 'Shape_Area']].merge(wetlands_df[['ntaname', 'Shape_Area_wetlands_sum']], left_on='NTAName', right_on='ntaname', how='left')

# export the merged result to an Excel file
merged_df.to_excel('nta_wetlands_nta2020.xlsx', index=False)

print("Exported NTA wetlands summary to 'nta_wetlands_nta2020.xlsx'.")


Exported NTA wetlands summary to 'nta_wetlands_nta2020.xlsx'.


#### Urban Animal Response

In [ ]:
urban_animal_df = pd.read_excel('parkproperties_Statistics_2020ntacounting_condition_TableToExcel.xlsx')

In [ ]:
# Identify missing NTAs in the building dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(urban_animal_df['ntaname'])

# Create a dataframe to display missing NTAs
missing_ntas_df = pd.DataFrame(list(missing_ntas), columns=['Missing_NTAs'])
missing_ntas_df.head()

,Missing_NTAs
0,LaGuardia Airport
1,Snug Harbor
2,Kissena Park
3,Woodlawn Cemetery
4,The Battery-Governors Island-Ellis Island-Libe...


In [ ]:
# Assign values to LSTSTATYPE
# Define the mapping: 1 for constructed/permitted/under construction, 0 for marked/blanks
value_mapping = {
    'Unhealthy': 1,
    'N/A': 1,
    'Injured': 1,
    'Healthy': 0.5,
    'DOA': 1,
    '': 0  # Blank rows
}

#Unhealthy
#N/A
#Injured
#Healthy
#DOA (<- means dead on arrival)
#blanks

In [ ]:
# Apply the mapping to the building dataset
urban_animal_df['Value'] = urban_animal_df['Animal Condition'].map(value_mapping)

In [ ]:
# Calculate vulnerability value for each NTA
# Multiply 'Value' by 'FREQUENCY' and group by 'Geography' (NTA name) to sum the values
urban_animal_df['Weighted_Value'] = urban_animal_df['Value'] * urban_animal_df['FREQUENCY']

# Group by Geography to calculate the total vulnerability value for each NTA
vulnerability_df = urban_animal_df.groupby('ntaname')['Weighted_Value'].sum().reset_index()

# Merge the vulnerability values back with the NTA dataset to include all NTAs
final_df = pd.merge(nta_2020_df[['NTAName']], vulnerability_df, left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing vulnerability values with 0 (in case an NTA has no building data)
final_df['Weighted_Value'].fillna(0, inplace=True)

# Export the final result to Excel
final_df.to_excel('Urban_Animal_recalculated_values_nta2020.xlsx', index=False)

print("Vulnerability values calculated and exported to 'Urban_Animal_recalculated_values_nta2020.xlsx'.")

Vulnerability values calculated and exported to 'Urban_Animal_recalculated_values_nta2020.xlsx'.


<ipython-input-20-f358203e92b8>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Weighted_Value'].fillna(0, inplace=True)


#### Park

In [ ]:
%cd /content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified

/content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified


In [ ]:
park_df = pd.read_excel('park_wild_2020nta_Project_TableToExcel.xlsx')

In [ ]:
#  Identify missing NTAs in the building dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(park_df['ntaname'])

# Create a dataframe to display missing NTAs
missing_ntas_df = pd.DataFrame(list(missing_ntas), columns=['Missing_NTAs'])
missing_ntas_df.head()

,Missing_NTAs
0,Astoria (East)-Woodside (North)
1,South Jamaica
2,Fordham Heights
3,Dyker Beach Park
4,Brighton Beach


In [ ]:
import pandas as pd

# Sum the park areas within each NTA
park_area_sum_df = park_df.groupby('ntaname').agg(
    total_park_area=('Shape_Area', 'sum'),
    nta_area=('Shape_Area_nta', 'first')  # Assuming 'Shape_Area_nta' is the same for all rows in the same NTA
).reset_index()

In [ ]:
# Merge the total park area and nta_area with the NTA dataset
final_df = pd.merge(nta_2020_df[['NTAName']], park_area_sum_df, left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing total park areas with 0 (if an NTA has no parks listed in the park dataset)
final_df['total_park_area'].fillna(0, inplace=True)

#  Export the results to an Excel file
final_df.to_excel('nta_park_area_summary.xlsx', index=False)

print("Park area summaries calculated and exported to 'nta_park_area_summary.xlsx'.")

Park area summaries calculated and exported to 'nta_park_area_summary.xlsx'.


<ipython-input-9-03eb4e38bdd7>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['total_park_area'].fillna(0, inplace=True)


#### Cooling sites

In [ ]:
coolingsites_df = pd.read_excel('Cool_It__NYC_2020_Statistics2_TableToExcel.xlsx')

In [ ]:
# Identify missing NTAs in the building dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(coolingsites_df['ntaname'])

# Create a dataframe to display missing NTAs
missing_ntas_df = pd.DataFrame(list(missing_ntas), columns=['Missing_NTAs'])
missing_ntas_df.head()

,Missing_NTAs
0,St. George-New Brighton
1,Sunnyside
2,Spring Creek Park
3,Astoria Park
4,South Richmond Hill


In [ ]:
#  Assign values to LSTSTATYPE
# Define the mapping: 1 for constructed/permitted/under construction, 0 for marked/blanks
value_mapping = {
    'Activated': 1,
    'Broken': 0.25,
    'Not Yet Activated': 0.75,
    '': 0  # Blank rows
}

#Activated
#Broken
#NotYetActivated
#UnderConstruction

In [ ]:
# Apply the mapping to the building dataset
coolingsites_df['Value'] = coolingsites_df['Status'].map(value_mapping)

In [ ]:
#  Calculate vulnerability value for each NTA
# Multiply 'Value' by 'FREQUENCY' and group by 'Geography' (NTA name) to sum the values
coolingsites_df['Weighted_Value'] = coolingsites_df['Value'] * coolingsites_df['FREQUENCY']

# Group by Geography to calculate the total vulnerability value for each NTA
vulnerability_df = coolingsites_df.groupby('ntaname')['Weighted_Value'].sum().reset_index()

# Merge the vulnerability values back with the NTA dataset to include all NTAs
final_df = pd.merge(nta_2020_df[['NTAName']], vulnerability_df, left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing vulnerability values with 0 (in case an NTA has no building data)
final_df['Weighted_Value'].fillna(0, inplace=True)

#  Export the final result to Excel
final_df.to_excel('Coolingsites_recalculated_values_nta2020.xlsx', index=False)

print("Vulnerability values calculated and exported to 'Coolingsites_recalculated_values_nta2020.xlsx'.")

Vulnerability values calculated and exported to 'Coolingsites_recalculated_values_nta2020.xlsx'.


<ipython-input-34-56156decc7f6>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Weighted_Value'].fillna(0, inplace=True)


#### Drinking Fountains

In [ ]:
drinkingfountain_df = pd.read_excel('Cool_It__NYC_2020_DrinkingFountain_Statistics_TableToExcel.xlsx')

In [ ]:
# Identify missing NTAs in the building dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(drinkingfountain_df['ntaname'])

# Create a dataframe to display missing NTAs
missing_ntas_df = pd.DataFrame(list(missing_ntas), columns=['Missing_NTAs'])
missing_ntas_df.head()

,Missing_NTAs
0,Middle Village Cemetery
1,Shirley Chisholm State Park
2,Spring Creek Park
3,Barren Island-Floyd Bennett Field
4,Richmond Hill


In [ ]:
# Step 2: Assign values to LSTSTATYPE
# Define the mapping: 1 for constructed/permitted/under construction, 0 for marked/blanks
value_mapping = {
    'Activated': 1,
    'Broken': 0.25,
    'NotYetActivated': 0.75,
    'UnderConstruction': 0.5,
    '': 0  # Blank rows
}

#Activated
#Broken
#NotYetActivated
#UnderConstruction

In [ ]:
# Apply the mapping to the building dataset
drinkingfountain_df['Value'] = drinkingfountain_df['DF Activated'].map(value_mapping)

In [ ]:
# Calculate vulnerability value for each NTA
# Multiply 'Value' by 'FREQUENCY' and group by 'Geography' (NTA name) to sum the values
drinkingfountain_df['Weighted_Value'] = drinkingfountain_df['Value'] * drinkingfountain_df['FREQUENCY']

# Group by Geography to calculate the total vulnerability value for each NTA
vulnerability_df = drinkingfountain_df.groupby('ntaname')['Weighted_Value'].sum().reset_index()

# Merge the vulnerability values back with the NTA dataset to include all NTAs
final_df = pd.merge(nta_2020_df[['NTAName']], vulnerability_df, left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing vulnerability values with 0 (in case an NTA has no building data)
final_df['Weighted_Value'].fillna(0, inplace=True)

# Export the final result to Excel
final_df.to_excel('DrinkingFountain_recalculated_values_nta2020.xlsx', index=False)

print("Vulnerability values calculated and exported to 'DrinkingFountain_recalculated_values_nta2020.xlsx'.")

Vulnerability values calculated and exported to 'DrinkingFountain_recalculated_values_nta2020.xlsx'.


<ipython-input-28-95fe40e51132>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Weighted_Value'].fillna(0, inplace=True)


#### Spray Shower

In [ ]:
sprayshower_df = pd.read_excel('Cool_It__NYC_2020_Statistics_sprayshower_TableToExcel.xlsx')

In [ ]:
# Identify missing NTAs in the building dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(sprayshower_df['ntaname'])

# Create a dataframe to display missing NTAs
missing_ntas_df = pd.DataFrame(list(missing_ntas), columns=['Missing_NTAs'])
missing_ntas_df.head()

,Missing_NTAs
0,Middle Village Cemetery
1,East Flatbush-Erasmus
2,Shirley Chisholm State Park
3,Spring Creek Park
4,Freshkills Park (North)


In [ ]:
# Assign values to LSTSTATYPE
# Define the mapping: 1 for constructed/permitted/under construction, 0 for marked/blanks
value_mapping = {
    'Activated': 1,
    'Broken': 0.25,
    'Not Yet Activated': 0.75,
    'Under Construction': 0.5,
    '': 0  # Blank rows
}


#Activated
#Broken
#Not Yet Activated
#Under Construction

In [ ]:
# Apply the mapping to the building dataset
sprayshower_df['Value'] = sprayshower_df['Status'].map(value_mapping)

In [ ]:
# Calculate vulnerability value for each NTA
# Multiply 'Value' by 'FREQUENCY' and group by 'Geography' (NTA name) to sum the values
sprayshower_df['Weighted_Value'] = sprayshower_df['Value'] * sprayshower_df['FREQUENCY']

# Group by Geography to calculate the total vulnerability value for each NTA
vulnerability_df = sprayshower_df.groupby('ntaname')['Weighted_Value'].sum().reset_index()

# Merge the vulnerability values back with the NTA dataset to include all NTAs
final_df = pd.merge(nta_2020_df[['NTAName']], vulnerability_df, left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing vulnerability values with 0 (in case an NTA has no building data)
final_df['Weighted_Value'].fillna(0, inplace=True)

# Step 4: Export the final result to Excel
final_df.to_excel('Sprayshower_recalculated_values_nta2020.xlsx', index=False)


print("Vulnerability values calculated and exported to 'Sprayshower_recalculated_values_nta2020.xlsx'.")

Vulnerability values calculated and exported to 'Sprayshower_recalculated_values_nta2020.xlsx'.


<ipython-input-21-f1239716af23>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Weighted_Value'].fillna(0, inplace=True)


#### GI (green infrastructure)

In [ ]:
%cd /content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified

/content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified


In [ ]:
gi_df = pd.read_excel('DEP_GI_Assets_Pub_Statistics_2020nta_TableToExcel.xlsx')

In [ ]:
# Identify missing NTAs in the building dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(gi_df['ntaname'])

# Create a dataframe to display missing NTAs
missing_ntas_df = pd.DataFrame(list(missing_ntas), columns=['Missing_NTAs'])
missing_ntas_df.head()

,Missing_NTAs
0,Middle Village Cemetery
1,Alley Pond Park
2,Shirley Chisholm State Park
3,St. George-New Brighton
4,Spring Creek Park


In [ ]:
# Assign values to LSTSTATYPE
# Define the mapping: 1 for constructed/permitted/under construction, 0 for marked/blanks
value_mapping = {
    '100% Design Accepted': 0.5,
    '100% Design Submitted': 0.5,
    '90% Design Accepted': 0.25,
    '90% Design Submitted': 0.25,
    'Constructed': 1,
    'Constructed (Full Maintenance)': 1,
    'Constructed (In Guarantee)': 1,
    'In Construction': 0.75,
    '': 0  # Blank rows
}

#''
#100% Design Accepted
#100% Design Submitted
#90% Design Accepted
#90% Design Submitted
#Constructed
#Constructed (Full Maintenance)
#Constructed (In Guarantee)
#In Construction

In [ ]:
# Apply the mapping to the building dataset
gi_df['Value'] = gi_df['Status'].map(value_mapping)

In [ ]:
# Calculate vulnerability value for each NTA
# Multiply 'Value' by 'FREQUENCY' and group by 'Geography' (NTA name) to sum the values
gi_df['Weighted_Value'] = gi_df['Value'] * gi_df['FREQUENCY']

# Group by Geography to calculate the total vulnerability value for each NTA
vulnerability_df = gi_df.groupby('ntaname')['Weighted_Value'].sum().reset_index()

# Merge the vulnerability values back with the NTA dataset to include all NTAs
final_df = pd.merge(nta_2020_df[['NTAName']], vulnerability_df, left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing vulnerability values with 0 (in case an NTA has no building data)
final_df['Weighted_Value'].fillna(0, inplace=True)

# Export the final result to Excel
final_df.to_excel('GI_recalculated_values_nta2020.xlsx', index=False)

print("Vulnerability values calculated and exported to 'GI_recalculated_values.xlsx'.")

Vulnerability values calculated and exported to 'GI_recalculated_values.xlsx'.


<ipython-input-10-66da04351b4e>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Weighted_Value'].fillna(0, inplace=True)


#### Building

In [ ]:
%cd /content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified

/content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified


In [ ]:
building_df = pd.read_excel('building_Select1_Statistics_TableToExcel.xlsx')  # Building dataset with 'LSTSTATYPE' and 'FREQUENCY'

In [ ]:
# Identify missing NTAs in the building dataset
missing_ntas = set(nta_2020_df['NTAName']) - set(building_df['ntaname'])

# Create a dataframe to display missing NTAs
missing_ntas_df = pd.DataFrame(list(missing_ntas), columns=['Missing_NTAs'])
missing_ntas_df.head()

,Missing_NTAs
0,Middle Village Cemetery
1,Alley Pond Park
2,Shirley Chisholm State Park
3,Spring Creek Park
4,Freshkills Park (North)


In [ ]:
#  Assign values to LSTSTATYPE
# Define the mapping: 1 for constructed/permitted/under construction, 0 for marked/blanks
value_mapping = {
    'Constructed': 1,
    'Permitted': 0.5,
    'Under Construction': 0.75,
    'Marked for Construction': 0.25,
    '': 0  # Blank rows
}

In [ ]:
# Apply the mapping to the building dataset
building_df['Value'] = building_df['LSTSTATYPE'].map(value_mapping)

In [ ]:
# Calculate vulnerability value for each NTA
# Multiply 'Value' by 'FREQUENCY' and group by 'Geography' (NTA name) to sum the values
building_df['Weighted_Value'] = building_df['Value'] * building_df['FREQUENCY']

# Group by Geography to calculate the total vulnerability value for each NTA
vulnerability_df = building_df.groupby('ntaname')['Weighted_Value'].sum().reset_index()

# Merge the vulnerability values back with the NTA dataset to include all NTAs
final_df = pd.merge(nta_2020_df[['NTAName']], vulnerability_df, left_on='NTAName', right_on='ntaname', how='left')

# Fill any missing vulnerability values with 0 (in case an NTA has no building data)
final_df['Weighted_Value'].fillna(0, inplace=True)

# Export the final result to Excel
final_df.to_excel('building_recalculated_values_nta2020.xlsx', index=False)


print("Vulnerability values calculated and exported to 'building_recalculated_values_nta2020.xlsx'.")

Vulnerability values calculated and exported to 'building_recalculated_values_nta2020.xlsx'.


<ipython-input-15-0f086da39f21>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Weighted_Value'].fillna(0, inplace=True)


In [ ]:
building_df.head()

,OBJECTID,ntaname,nta2020,LSTSTATYPE,FREQUENCY,COUNT_building_Select1_AddSpatialJoin.ntaname,Value,Weighted_Value
0,1,NaN,NaN,Constructed,2,NaN,1.0,2.0
1,2,Allerton,BX1104,Constructed,14,14.0,1.0,14.0
2,3,Astoria (Central),QN0103,Constructed,25,25.0,1.0,25.0
3,4,Astoria (East)-Woodside (North),QN0104,Constructed,7,7.0,1.0,7.0
4,5,Astoria (North)-Ditmars-Steinway,QN0101,Constructed,14,14.0,1.0,14.0


In [ ]:
vulnerability_df.head()

,nta2020,Weighted_Value
0,BK0101,23.0
1,BK0102,117.0
2,BK0103,131.0
3,BK0104,52.0
4,BK0201,117.0


#### Independent Living Difficulty (adults)

In [ ]:
ind_dif_2020_df = pd.read_csv('NYC EH Data Portal - Independent living difficulty (adults) (filtered)_2020 (num).csv')

In [ ]:
# standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
ind_dif_2020_df['Geography'] = ind_dif_2020_df['Geography'].str.strip()

In [ ]:
# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(ind_dif_2020_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
# Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
ind_dif_2020_df_avg = ind_dif_2020_df.groupby('Geography').apply(
    lambda x: x['Percent'].mean()
).reset_index(name='Average_Individual_Level_Dif_Percent')

<ipython-input-16-f5a91cf0ce23>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ind_dif_2020_df_avg = ind_dif_2020_df.groupby('Geography').apply(


In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, ind_dif_2020_df_avg, left_on='NTAName', right_on='Geography', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('ind_dif_2020_df.xlsx', index=False)

print("Export completed: 'ind_dif_2020_df.xlsx'")

Export completed: 'ind_dif_2020_df.xlsx'


#### Graduated High-school

In [ ]:
grad_high_2020_df = pd.read_csv('NYC EH Data Portal - Graduated high school (filtered)_2020 (num).csv')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
grad_high_2020_df['Geography'] = grad_high_2020_df['Geography'].str.strip()

In [ ]:
# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(grad_high_2020_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
# Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
grad_high_2020_df_avg = grad_high_2020_df.groupby('Geography').apply(
    lambda x: x['Percent'].mean()
).reset_index(name='Average_Grad_High_Percent')

<ipython-input-9-ffd62a158c82>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grad_high_2020_df_avg = grad_high_2020_df.groupby('Geography').apply(


In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, grad_high_2020_df_avg, left_on='NTAName', right_on='Geography', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('grad_high_2020_df.xlsx', index=False)

print("Export completed: 'grad_high_2020_df.xlsx'")

Export completed: 'grad_high_2020_df.xlsx'


#### Limited English

In [ ]:
limited_english_df = pd.read_csv('NYC EH Data Portal - Limited English (filtered)_2020 (num).csv')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
limited_english_df['Geography'] = limited_english_df['Geography'].str.strip()

In [ ]:
# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(limited_english_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
# Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
limited_english_df_avg = limited_english_df.groupby('Geography').apply(
    lambda x: x['Percent'].mean()
).reset_index(name='Average_LE_Percent')

<ipython-input-54-c1a24ce4e912>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  limited_english_df_avg = limited_english_df.groupby('Geography').apply(


In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, limited_english_df_avg, left_on='NTAName', right_on='Geography', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('limited_english_2020_df.xlsx', index=False)

print("Export completed: 'limited_english_2020_df.xlsx'")

Export completed: 'limited_english_2020_df.xlsx'


#### Older Adults living alone

In [ ]:
oa_alone_df = pd.read_csv('NYC EH Data Portal - Older adults living alone (filtered)_2020 (num).csv')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
oa_alone_df['Geography'] = oa_alone_df['Geography'].str.strip()

In [ ]:
#  Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(oa_alone_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
# Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
oa_alone_df_avg = oa_alone_df.groupby('Geography').apply(
    lambda x: x['Percent'].mean()
).reset_index(name='Average_Older Adults (percent)')

<ipython-input-59-69f1d85347d4>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  oa_alone_df_avg = oa_alone_df.groupby('Geography').apply(


In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, oa_alone_df_avg, left_on='NTAName', right_on='Geography', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('oa_alone_2020_df.xlsx', index=False)

print("Export completed: 'oa_alone_2020_df.xlsx'")

Export completed: 'oa_alone_2020_df.xlsx'


#### Bicycle

In [ ]:
bicycle_df = pd.read_csv('NYC EH Data Portal - Commuting modes (filtered)_2020_bicycle - Data.csv')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
bicycle_df['Geography'] = bicycle_df['Geography'].str.strip()

In [ ]:
# find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(bicycle_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
# Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
bicycle_df_avg = bicycle_df.groupby('Geography').apply(
    lambda x: x['Bicycle (percent)'].mean()
).reset_index(name='Average_Bicycle (percent)')

<ipython-input-42-ce0262c9f23e>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bicycle_df_avg = bicycle_df.groupby('Geography').apply(


In [ ]:
#  Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, bicycle_df_avg, left_on='NTAName', right_on='Geography', how='left')

#  Export the final dataframe to an Excel file
merged_df.to_excel('bicycle_df.xlsx', index=False)

print("Export completed: 'bicycle_df.xlsx'")

Export completed: 'bicycle_df.xlsx'


#### Public transportation

In [ ]:
publictransportation_df = pd.read_csv('NYC EH Data Portal - Commuting modes (filtered)_2020_publictransportation - Data.csv')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
publictransportation_df['Geography'] = publictransportation_df['Geography'].str.strip()

In [ ]:
#  Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(publictransportation_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
#  Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
publictransportation_df_avg = publictransportation_df.groupby('Geography').apply(
    lambda x: x['Public transportation (percent)'].mean()
).reset_index(name='Average_Public transportation (percent)')

<ipython-input-35-8b04e51963ea>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  publictransportation_df_avg = publictransportation_df.groupby('Geography').apply(


In [ ]:
#  Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, publictransportation_df_avg, left_on='NTAName', right_on='Geography', how='left')

#  Export the final dataframe to an Excel file
merged_df.to_excel('publictransportation_df.xlsx', index=False)

print("Export completed: 'publictransportation_df.xlsx'")

Export completed: 'publictransportation_df.xlsx'


#### Car, truck, van

In [ ]:
car_truck_van_df = pd.read_csv('NYC EH Data Portal - Commuting modes (filtered)_2020_cartruckvan - Data.csv')

In [ ]:
#  Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
car_truck_van_df['Geography'] = car_truck_van_df['Geography'].str.strip()

In [ ]:
# find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(car_truck_van_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
#  Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
car_truck_van_df_avg = car_truck_van_df.groupby('Geography').apply(
    lambda x: x['Car, truck, or van (percent)'].mean()
).reset_index(name='Average_Car, truck, or van (percent)')

<ipython-input-27-87a1bd6a82da>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  car_truck_van_df_avg = car_truck_van_df.groupby('Geography').apply(


In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, car_truck_van_df_avg, left_on='NTAName', right_on='Geography', how='left')

#  Export the final dataframe to an Excel file
merged_df.to_excel('car_truck_van_2020_df.xlsx', index=False)

print("Export completed: 'car_truck_van_2020_df.xlsx'")

Export completed: 'car_truck_van_2020_df.xlsx'


#### walking

In [ ]:
%cd /content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified

/content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified


In [ ]:
walked_df = pd.read_csv('NYC EH Data Portal - Commuting modes (filtered)_2020_walked - Data.csv')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
walked_df['Geography'] = walked_df['Geography'].str.strip()

In [ ]:
# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(walked_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
# Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
walked_df_avg = walked_df.groupby('Geography').apply(
    lambda x: x['Walked (percent)'].mean()
).reset_index(name='Average_Walked (percent)')

<ipython-input-22-eb3308934dd4>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  walked_df_avg = walked_df.groupby('Geography').apply(


In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, walked_df_avg, left_on='NTAName', right_on='Geography', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('walked_2020_df.xlsx', index=False)

print("Export completed: 'walked_2020_df.xlsx'")

Export completed: 'walked_2020_df.xlsx'


#### citi bike stations

In [ ]:
citibike_stations_df = pd.read_csv('NYC EH Data Portal - Citi Bike stations (filtered) (num).csv')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
citibike_stations_df['Geography'] = citibike_stations_df['Geography'].str.strip()

In [ ]:
# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(citibike_stations_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, citibike_stations_df, left_on='NTAName', right_on='Geography', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('citibike_stations_2020_df.xlsx', index=False)

print("Export completed: 'citibike_stations_2020_df.xlsx'")

Export completed: 'citibike_stations_2020_df.xlsx'


#### citibike availability

In [ ]:
citibike_df = pd.read_csv('NYC EH Data Portal - Average Citi Bike availability (filtered) (num).csv')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
citibike_df['Geography'] = citibike_df['Geography'].str.strip()

In [ ]:
# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(citibike_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
#Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, citibike_df, left_on='NTAName', right_on='Geography', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('citibike_2020_df.xlsx', index=False)

print("Export completed: 'citibike_2020_df.xlsx'")

Export completed: 'citibike_2020_df.xlsx'


#### arrest

In [ ]:
arrest_2020_df = pd.read_excel('arrest_Statistics_2020_TableToExcel.xlsx')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
arrest_2020_df['ntaname'] = arrest_2020_df['ntaname'].str.strip()

In [ ]:
# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(arrest_2020_df['ntaname'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Calvert Vaux Park', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Hart Island', 'Mount Olivet & All Faiths Cemeteries', 'Fort Totten', 'Mount Hebron & Cedar Grove Cemeteries', 'Spring Creek Park', 'Montefiore Cemetery', 'Hoffman & Swinburne Islands']


In [ ]:
len(missing_ntas_list)

11

In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, arrest_2020_df, left_on='NTAName', right_on='ntaname', how='left')

# export the final dataframe to an Excel file
merged_df.to_excel('arrest_nta2020.xlsx', index=False)

print("Export completed: 'arrest_nta2020.xlsx'")

Export completed: 'arrest_nta2020.xlsx'


#### food scrap

In [ ]:
%cd /content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified

/content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified


In [ ]:
foodscrap_2020_df = pd.read_excel('food_scrap_2020_Statistics_TableToExcel.xlsx')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
foodscrap_2020_df['ntaname'] = foodscrap_2020_df['ntaname'].str.strip()

In [ ]:
# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(foodscrap_2020_df['ntaname'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['South Williamsburg', 'Brooklyn Heights', 'Clinton Hill', 'Brooklyn Navy Yard', 'Spring Creek-Starrett City', 'East New York-City Line', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Bath Beach', 'Gravesend (West)', 'Sunset Park (East)-Borough Park (West)', 'Borough Park', 'Mapleton-Midwood (West)', 'Gravesend (South)', 'Brighton Beach', 'Calvert Vaux Park', 'Midwood', 'Gravesend (East)-Homecrest', 'Madison', 'Sheepshead Bay-Manhattan Beach-Gerritsen Beach', 'East Flatbush-Farragut', 'Holy Cross Cemetery', 'Flatlands', 'Canarsie', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'Hunts Point', 'North & South Brother Islands', 'Claremont Village-Claremont (East)', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Pa

In [ ]:
len(missing_ntas_list)

127

In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, foodscrap_2020_df, left_on='NTAName', right_on='ntaname', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('foodscrap_nta2020.xlsx', index=False)

print("Export completed: 'foodscrap_nta2020.xlsx'")

Export completed: 'foodscrap_nta2020.xlsx'


#### mta

In [ ]:
%cd /content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified

/content/drive/MyDrive/Thesis/Fall_2024/Data/Datasets/Modified


In [ ]:
mta2020 = pd.read_excel('mta_subway_Statistics2020_TableToExcel.xlsx')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
mta2020['ntaname'] = mta2020['ntaname'].str.strip()

In [ ]:
# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(mta2020['ntaname'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Spring Creek-Starrett City', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Dyker Heights', 'Fort Hamilton', 'Dyker Beach Park', 'Bath Beach', 'Calvert Vaux Park', 'East Flatbush-Rugby', 'East Flatbush-Remsen Village', 'Holy Cross Cemetery', 'Flatlands', 'Marine Park-Mill Basin-Bergen Beach', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'Hunts Point', 'North & South Brother Islands', 'Morrisania', 'Claremont Village-Claremont (East)', 'Crotona Park', 'Highbridge', 'Claremont Park', 'Tremont', 'Belmont', 'Kingsbridge Heights-Van Cortlandt Village', 'Riverdale-Spuyten Duyvil', 'Soundview-Clason Point', 'Parkchester', 'Soundview Park', 'Throgs Neck-Schuylerville', 'Co-op City', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery'

In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, mta2020, left_on='NTAName', right_on='ntaname', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('mta_nta2020.xlsx', index=False)

print("Export completed: 'mta_nta2020.xlsx")

Export completed: 'mta_nta2020.xlsx


#### farmer's market

In [ ]:
farmer_market_df = pd.read_csv('NYC EH Data Portal - Farmers markets (filtered).csv')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
farmer_market_df['Geography'] = farmer_market_df['Geography'].str.strip()

In [ ]:
# find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(farmer_market_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
len(missing_ntas_list)

65

In [ ]:
# calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
farmer_market_df_avg = farmer_market_df.groupby('Geography').apply(
    lambda x: x['Density (year-round) per square mile'].mean()
).reset_index(name='Average_Density (year-round) per square mile')

<ipython-input-9-c74e96e068db>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  farmer_market_df_avg = farmer_market_df.groupby('Geography').apply(


In [ ]:
# merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, farmer_market_df_avg, left_on='NTAName', right_on='Geography', how='left')

# eport the final dataframe to an Excel file
merged_df.to_excel('farmer_market_df_avg_avg_nta2020_yearround.xlsx', index=False)

print("Export completed: 'farmer_market_df_avg_avg_nta2020_yearround.xlsx")

Export completed: 'farmer_market_df_avg_avg_nta2020_yearround.xlsx


In [ ]:
# calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
farmer_market_df_avg = farmer_market_df.groupby('Geography').apply(
    lambda x: x['Density per square mile'].mean()
).reset_index(name='Average_Density per square mile')

<ipython-input-11-f0bf5eb9d248>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  farmer_market_df_avg = farmer_market_df.groupby('Geography').apply(


In [ ]:
# merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, farmer_market_df_avg, left_on='NTAName', right_on='Geography', how='left')

# export the final dataframe to an Excel file
merged_df.to_excel('farmer_market_df_avg_avg_nta2020_den.xlsx', index=False)

print("Export completed: 'farmer_market_df_avg_avg_nta2020_den.xlsx")

Export completed: 'farmer_market_df_avg_avg_nta2020_den.xlsx


#### litter basket

In [ ]:
litter_basket_df = pd.read_csv('NYC EH Data Portal - Litter basket coverage (filtered) - (num).csv')

In [ ]:
# Step 1: Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
litter_basket_df['Geography'] = litter_basket_df['Geography'].str.strip()

In [ ]:
# Step 2: Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(litter_basket_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
len(missing_ntas_list)

65

In [ ]:
# Step 3: Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
litter_basket_df_avg = litter_basket_df.groupby('Geography').apply(
    lambda x: x['Density (non-wire) per square mile'].mean()
).reset_index(name='Average_Density (non-wire) per square mile')

<ipython-input-22-d6a3253e5bc2>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  litter_basket_df_avg = litter_basket_df.groupby('Geography').apply(


In [ ]:
# Step 3: Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
litter_basket_df_avg = litter_basket_df.groupby('Geography').apply(
    lambda x: x['Density per square mile'].mean()
).reset_index(name='Average_Density per square mile')

<ipython-input-24-7f9d6452563f>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  litter_basket_df_avg = litter_basket_df.groupby('Geography').apply(


In [ ]:
# Step 3: Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
litter_basket_df_avg = litter_basket_df.groupby('Geography').apply(
    lambda x: x['Percent (non-wire)'].mean()
).reset_index(name='Average_Percent (non-wire)')

<ipython-input-19-001d7002c0e7>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  litter_basket_df_avg = litter_basket_df.groupby('Geography').apply(


In [ ]:
# Step 4: Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, litter_basket_df_avg, left_on='NTAName', right_on='Geography', how='left')

# Step 5: Export the final dataframe to an Excel file
merged_df.to_excel('litter_basket_df_avg_nta2020_density.xlsx', index=False)

print("Export completed: 'litter_basket_df_avg_df_avg.xlsx")

Export completed: 'litter_basket_df_avg_df_avg.xlsx


Limited English

In [ ]:
#nta_df = pd.read_csv('nynta.csv')  # NTA dataset
limited_english_df = pd.read_csv('NYC EH Data Portal - Limited English (filtered)_2010nta - Sheet1.csv')

In [ ]:
# standardize column names for comparison
nta_df['NTAName'] = nta_df['NTAName'].str.strip()
limited_english_df['Geography'] = limited_english_df['Geography'].str.strip()

In [ ]:
# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_df[~nta_df['NTAName'].isin(limited_english_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in limited_english_df under data: {missing_ntas_list}")

Missing NTA Names in limited_english_df under data: ['Rikers Island', 'park-cemetery-etc-Staten Island', 'park-cemetery-etc-Bronx', 'park-cemetery-etc-Manhattan', 'park-cemetery-etc-Brooklyn', 'park-cemetery-etc-Queens', 'Airport']


In [ ]:
# Step 3: Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
limited_english_df_avg = limited_english_df.groupby('Geography').apply(
    lambda x: x['Percent'].mean()
).reset_index(name='AveragePercent')

In [ ]:
# merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, neigh_pov_2020nta_df_avg, left_on='NTAName', right_on='Geography', how='left')

# export the final dataframe to an Excel file
merged_df.to_excel('nta_neigh_pov_2020nta_df_avg .xlsx', index=False)

print("Export completed: 'nta_neigh_pov_2020nta_df_avg.xlsx")

Export completed: 'nta_neigh_pov_2020nta_df_avg.xlsx


2020 child pov under 5

In [ ]:
nta_2020_df = pd.read_csv('2020nta.csv')

In [ ]:
child_pov_under5_df = pd.read_csv('NYC EH Data Portal - Child poverty_nta2020 (under age 5) (filtered) (num).csv')

In [ ]:
# Standardize column names for comparison
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
child_pov_under5_df['Geography'] = child_pov_under5_df['Geography'].str.strip()

In [ ]:
# missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(child_pov_under5_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in child_pov_under5_df under data: {missing_ntas_list}")

Missing NTA Names in child_pov_under5_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)

In [ ]:
len(missing_ntas_list)

65

In [ ]:
# Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
child_pov_under5_avg = child_pov_under5_df.groupby('Geography').apply(
    lambda x: x['Percent'].mean()
).reset_index(name='AveragePercent')

In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, child_pov_under5_avg, left_on='NTAName', right_on='Geography', how='left')

# export the final dataframe to an Excel file
merged_df.to_excel('nta_child_pov_under5_avg .xlsx', index=False)

print("Export completed: 'nta_child_pov_under5_avg.xlsx")

Export completed: 'nta_child_pov_under5_avg.xlsx


2020 unemployment

In [ ]:
unemployment_2020_df = pd.read_csv('NYC EH Data Portal - Unemployment_nta2020 (filtered) - Sheet1.csv')

In [ ]:
nta_2020_df['NTAName'] = nta_2020_df['NTAName'].str.strip()
unemployment_2020_df['Geography'] = unemployment_2020_df['Geography'].str.strip()

In [ ]:
# Find missing NTA names in the rent-burdened households file
missing_ntas = nta_2020_df[~nta_2020_df['NTAName'].isin(unemployment_2020_df['Geography'])]
missing_ntas_list = missing_ntas['NTAName'].tolist()

# Print missing NTA names
print(f"Missing NTA Names in eviction_df under data: {missing_ntas_list}")

Missing NTA Names in eviction_df under data: ['Brooklyn Navy Yard', 'The Evergreens Cemetery', 'Highland Park-Cypress Hills Cemeteries (South)', 'Green-Wood Cemetery', 'Lincoln Terrace Park', 'Fort Hamilton', 'Dyker Beach Park', 'Calvert Vaux Park', 'Holy Cross Cemetery', 'Marine Park-Plumb Island', 'McGuire Fields', 'Canarsie Park & Pier', 'Prospect Park', 'Barren Island-Floyd Bennett Field', 'Jamaica Bay (West)', 'Shirley Chisholm State Park', 'North & South Brother Islands', 'Crotona Park', 'Yankee Stadium-Macombs Dam Park', 'Claremont Park', 'Soundview Park', 'Hart Island', 'Ferry Point Park-St. Raymond Cemetery', 'Hutchinson Metro Center', 'Woodlawn Cemetery', 'Van Cortlandt Park', 'Bronx Park', 'Pelham Bay Park', 'The Battery-Governors Island-Ellis Island-Liberty Island', 'United Nations', "Randall's Island", 'Highbridge Park', 'Inwood Hill Park', 'Central Park', 'Rikers Island', 'Sunnyside Yards (North)', "St. Michael's Cemetery", 'Astoria Park', 'Sunnyside Yards (South)', 'Calv

In [ ]:
len(missing_ntas_list)

65

In [ ]:
# Calculate the average percent for each NTA based on available time periods
# Group by Geography (NTA) and calculate the average percent
unemployment_2020_df_avg = unemployment_2020_df.groupby('Geography').apply(
    lambda x: x['Percent'].mean()
).reset_index(name='AveragePercent')

In [ ]:
# Merge the average percent data with the NTA data
merged_df = pd.merge(nta_2020_df, unemployment_2020_df_avg, left_on='NTAName', right_on='Geography', how='left')

# Export the final dataframe to an Excel file
merged_df.to_excel('nta_unemployment_2020_df_avg .xlsx', index=False)

print("Export completed: 'nta_unemployment_2020_df_avg.xlsx")

Export completed: 'nta_unemployment_2020_df_avg.xlsx
